In [1]:
import csv
from collections import defaultdict

In [2]:
# Common protocol number -> name mappings
# Extend if you expect other protocols
PROTO_MAP = {
    6: "tcp",
    17: "udp",
    1: "icmp",
}

In [3]:
def load_lookup_table(lookup_csv_path):
    lookup = {}
    with open(lookup_csv_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        for row in reader:
            # Each valid line should have at least 3 fields
            if len(row) < 3:
                continue
            dstport_str, proto_str, tag = row
            # Make it case-insensitive by lowercasing the keys
            key = (dstport_str.strip().lower(), proto_str.strip().lower())
            lookup[key] = tag.strip()
    return lookup

def parse_flow_log_line(line):
    parts = line.strip().split()
    # Check the number of fields to be a valid version 2 log line if not enough return None
    if len(parts) < 14:
        return None

    try:
        version = int(parts[0])
    except ValueError:
        # If the version is not an integer, skip
        return None
    # We only handle version 2 logs in this script
    if version != 2:       
        return None

    # Extract needed fields
    dst_port_str = parts[6]
    proto_num_str = parts[7]

    # Convert protocol number to text (tcp/udp/icmp) if known
    try:
        proto_num = int(proto_num_str)
        protocol_str = PROTO_MAP.get(proto_num, str(proto_num))
    except ValueError:
        protocol_str = proto_num_str  # fallback: raw string if not an integer

    return {
        "dstport": dst_port_str,
        "protocol": protocol_str,
    }

In [4]:
def main(flow_log_path, lookup_csv_path):
    # 1. Load lookup table
    lookup_dict = load_lookup_table(lookup_csv_path)

    # 2. Prepare counters
    tag_counts = defaultdict(int)         # tag -> count
    port_proto_counts = defaultdict(int)  # (dstport, protocol) -> count

    # 3. Read and parse the flow logs
    with open(flow_log_path, "r", encoding="utf-8") as f:
        for line in f:
            # Skip empty lines
            if not line.strip():
                continue

            parsed = parse_flow_log_line(line)
            if not parsed:
                continue  # skip invalid lines or non-version-2 lines

            # Standardize and pull out fields
            dstport = parsed["dstport"].strip()
            protocol = parsed["protocol"].strip().lower()  # ensure case-insensitive

            # Keep track of (dstport, protocol) count
            port_proto_key = (dstport, protocol)
            port_proto_counts[port_proto_key] += 1

            # Look for a tag in the lookup dict
            lookup_key = (dstport.lower(), protocol.lower())
            if lookup_key in lookup_dict:
                tag = lookup_dict[lookup_key]
            else:
                tag = "Untagged"

            # Increment the tag count
            tag_counts[tag] += 1

    # 4. Print results

    # --- Tag Counts ---
    print("Tag Counts:")
    print("Tag,Count")
    for tag, cnt in sorted(tag_counts.items()):
        print(f"{tag},{cnt}")
    print()  # blank line between sections

    # --- Port/Protocol Combination Counts ---
    print("Port/Protocol Combination Counts:")
    print("Port,Protocol,Count")
    for (port, proto), cnt in sorted(port_proto_counts.items()):
        print(f"{port},{proto},{cnt}")

In [5]:
if __name__ == "__main__":

    flow_log_file = 'flow_log.txt'
    lookup_csv_file = 'lookup.txt'
    main(flow_log_file, lookup_csv_file)


Tag Counts:
Tag,Count
DNS,258
HTTP,275
HTTPS,275
ICMP,302
IMAP,297
POP3,288
RDP,272
SMTP,319
SNMP,277
SSH,266
TELNET,302
TFTP,296
Untagged,94891
email,265
sv_P1,303
sv_P2,262
sv_P3,282
sv_P4,276
sv_P5,294

Port/Protocol Combination Counts:
Port,Protocol,Count
1,udp,1
10,icmp,1
100,icmp,1
100,udp,1
1000,icmp,1
10001,icmp,2
10001,tcp,2
10003,icmp,1
10003,tcp,1
10003,udp,1
10004,icmp,1
10004,tcp,1
10005,tcp,1
10006,icmp,2
10006,tcp,2
10007,icmp,1
10007,udp,1
10008,tcp,1
10008,udp,1
10009,icmp,1
10009,udp,1
1001,icmp,1
1001,tcp,2
1001,udp,1
10010,udp,2
10011,icmp,1
10012,tcp,2
10012,udp,1
10013,icmp,1
10013,tcp,1
10014,udp,1
10015,tcp,1
10015,udp,1
10017,tcp,2
10017,udp,1
10018,tcp,2
10019,icmp,1
10019,tcp,1
1002,tcp,1
10020,icmp,1
10020,tcp,2
10021,tcp,1
10023,tcp,1
10024,udp,1
10025,icmp,1
10025,udp,2
10026,tcp,2
10027,tcp,2
10027,udp,1
10028,icmp,2
10029,udp,1
10030,udp,2
10031,udp,1
10032,icmp,1
10032,tcp,1
10033,tcp,1
10034,icmp,1
10035,icmp,1
10035,tcp,1
10036,icmp,1
10036,tcp,1
1003

1176,icmp,1
1176,tcp,1
1176,udp,1
11761,udp,1
11762,tcp,1
11763,icmp,2
11764,icmp,1
11764,tcp,1
11764,udp,2
11766,icmp,3
11766,udp,3
11767,icmp,1
11767,tcp,1
11769,icmp,1
1177,icmp,1
1177,udp,1
11770,tcp,1
11771,icmp,1
11772,tcp,1
11772,udp,2
11773,icmp,1
11774,icmp,2
11774,tcp,1
11774,udp,1
11775,icmp,1
11778,icmp,1
11778,tcp,1
1178,tcp,1
11781,icmp,1
11781,tcp,1
11782,icmp,1
11783,tcp,1
11784,icmp,1
11784,udp,1
11786,tcp,1
11787,icmp,1
11788,icmp,1
11788,tcp,1
11789,tcp,2
1179,icmp,1
1179,tcp,1
11790,udp,1
11791,tcp,2
11791,udp,2
11792,icmp,1
11792,tcp,1
11793,icmp,1
11795,icmp,1
11795,tcp,1
11795,udp,2
11797,tcp,1
11797,udp,2
11798,icmp,1
11799,udp,1
118,tcp,1
118,udp,1
1180,icmp,1
1180,tcp,1
1180,udp,1
11800,icmp,1
11800,udp,1
11801,icmp,3
11801,udp,2
11802,icmp,1
11803,icmp,1
11803,tcp,1
11804,udp,1
11805,icmp,1
11806,icmp,1
11807,tcp,1
11807,udp,1
11808,icmp,1
11809,icmp,2
11809,tcp,1
11809,udp,1
1181,tcp,1
1181,udp,2
11810,tcp,1
11810,udp,1
11811,udp,2
11812,icmp,2
11812,tcp,3
1

13196,tcp,1
13196,udp,1
13197,icmp,2
13197,tcp,1
13198,icmp,1
13198,udp,1
13199,tcp,1
13199,udp,1
132,udp,1
1320,icmp,2
1320,tcp,1
13200,tcp,1
13200,udp,1
13201,icmp,1
13202,tcp,1
13203,udp,2
13204,icmp,1
13204,tcp,1
13205,icmp,1
13205,tcp,1
13206,icmp,2
13207,icmp,1
13207,udp,1
13208,tcp,1
13208,udp,1
1321,icmp,1
1321,udp,1
13211,tcp,1
13211,udp,2
13212,icmp,1
13212,udp,1
13213,icmp,1
13213,tcp,2
13213,udp,3
13214,icmp,2
13214,tcp,1
13215,icmp,3
13216,icmp,2
13217,icmp,1
13217,udp,1
13218,tcp,1
13218,udp,2
13219,icmp,1
13219,udp,2
13220,tcp,1
13221,icmp,1
13221,udp,3
13222,tcp,1
13222,udp,1
13223,icmp,1
13224,icmp,2
13225,tcp,1
13226,icmp,1
13226,tcp,3
13226,udp,2
13228,udp,2
1323,icmp,1
1323,tcp,2
1323,udp,2
13230,icmp,1
13230,udp,3
13231,tcp,2
13231,udp,1
13232,tcp,1
13232,udp,1
13233,tcp,1
13234,icmp,1
13235,icmp,1
13235,tcp,1
13238,tcp,1
13238,udp,2
13239,tcp,2
1324,tcp,1
13240,icmp,2
13241,udp,1
13243,udp,1
13244,icmp,2
13244,udp,1
13245,udp,1
13246,icmp,1
13248,icmp,1
13248,tcp,

14701,icmp,1
14701,udp,1
14702,icmp,1
14703,icmp,2
14705,tcp,1
14705,udp,1
14706,icmp,1
14706,tcp,1
14707,tcp,2
14708,udp,1
14709,icmp,1
1471,tcp,2
14710,icmp,1
14710,tcp,1
14710,udp,1
14711,icmp,1
14711,udp,1
14712,icmp,1
14712,udp,1
14714,icmp,1
14714,tcp,4
14715,tcp,1
14716,udp,1
14717,udp,1
14718,udp,1
14719,tcp,1
1472,udp,1
14720,icmp,1
14720,tcp,1
14721,tcp,1
14721,udp,1
1473,icmp,1
1473,tcp,1
14731,icmp,1
14731,tcp,2
14733,icmp,2
14735,tcp,1
14736,icmp,1
14736,tcp,1
14738,icmp,1
14738,tcp,3
14738,udp,1
14739,icmp,2
14739,tcp,1
1474,tcp,1
1474,udp,1
14740,tcp,2
14742,tcp,1
14742,udp,1
14743,tcp,2
14743,udp,2
14744,udp,2
14746,icmp,1
14747,icmp,2
14747,tcp,1
14749,icmp,2
14749,udp,1
1475,icmp,2
14750,udp,1
14751,tcp,1
14752,tcp,2
14754,tcp,1
14755,tcp,1
14756,tcp,2
14756,udp,1
14757,tcp,2
14758,icmp,1
14758,tcp,2
14759,icmp,1
14759,tcp,2
14759,udp,1
1476,icmp,2
14760,icmp,1
14760,tcp,1
14760,udp,1
14762,icmp,1
14762,udp,2
14763,icmp,1
14763,tcp,1
14764,tcp,1
14765,icmp,1
14767,icm

162,udp,1
1620,udp,1
16200,tcp,1
16202,tcp,1
16202,udp,1
16203,tcp,1
16203,udp,1
16204,tcp,1
16205,icmp,1
16205,tcp,1
16205,udp,1
16206,icmp,1
16207,icmp,1
16207,tcp,1
16207,udp,1
16208,udp,2
16209,icmp,2
16209,tcp,1
16209,udp,1
1621,icmp,2
1621,tcp,1
16210,tcp,2
16211,udp,1
16213,udp,1
16214,icmp,3
16214,tcp,1
16214,udp,1
16216,udp,1
16217,tcp,2
16218,icmp,1
16218,udp,1
1622,tcp,1
1622,udp,1
16220,udp,1
16221,tcp,1
16221,udp,2
16222,udp,1
16223,icmp,1
16224,icmp,1
16225,icmp,2
16225,tcp,1
16227,icmp,1
16228,tcp,1
16229,icmp,1
16229,tcp,1
16231,tcp,1
16232,icmp,2
16232,tcp,1
16233,tcp,1
16233,udp,1
16234,icmp,1
16234,tcp,1
16234,udp,1
16235,icmp,2
16235,udp,1
16236,tcp,1
16236,udp,1
16237,udp,1
16238,tcp,1
16239,icmp,1
1624,udp,1
16240,icmp,1
16243,udp,2
16244,icmp,1
16244,udp,2
16246,icmp,1
16246,udp,1
16247,tcp,1
16249,udp,1
1625,tcp,2
16253,udp,1
16254,tcp,1
16255,icmp,1
16257,icmp,2
16258,icmp,1
16259,icmp,1
16259,tcp,2
1626,icmp,1
16263,icmp,1
16263,tcp,1
16265,icmp,1
16265,tcp,1


...

62206,icmp,1
62206,udp,1
62207,tcp,1
62207,udp,1
62208,icmp,1
62208,tcp,1
62208,udp,1
62209,tcp,1
62209,udp,1
62211,icmp,1
62211,udp,1
62212,tcp,1
62213,tcp,1
62213,udp,1
62214,tcp,1
62215,icmp,1
62216,icmp,1
62218,icmp,1
62218,udp,1
62219,udp,1
6222,icmp,2
6222,tcp,2
62220,tcp,1
62221,udp,2
62223,icmp,1
62223,tcp,1
62224,udp,1
62225,udp,1
62226,tcp,2
62227,udp,2
62228,tcp,1
62229,icmp,1
62229,udp,1
62230,tcp,1
62231,udp,1
62232,udp,1
62233,icmp,1
62234,icmp,1
62234,tcp,2
62234,udp,1
62235,tcp,1
62236,icmp,2
62236,udp,2
62237,icmp,1
62237,tcp,1
62237,udp,1
62238,icmp,4
62239,icmp,1
6224,udp,2
62241,icmp,1
62242,icmp,1
62243,icmp,1
62244,icmp,1
62245,tcp,2
62245,udp,1
62246,tcp,3
62248,tcp,1
62249,tcp,1
6225,tcp,1
62250,icmp,1
62250,tcp,1
62250,udp,1
62251,icmp,2
62252,udp,1
62253,icmp,1
62253,udp,1
62254,tcp,1
62255,tcp,1
62255,udp,2
62256,tcp,1
62257,icmp,1
62257,tcp,2
62257,udp,1
62258,tcp,1
62259,icmp,1
62259,tcp,1
6226,icmp,1
6226,udp,1
62260,icmp,1
62262,icmp,1
62262,udp,1
62263,t

63667,icmp,1
63667,tcp,1
63668,icmp,1
63668,udp,2
63669,icmp,1
63670,tcp,1
63670,udp,1
63672,icmp,1
63673,tcp,1
63673,udp,1
63674,icmp,1
63674,udp,1
63675,udp,1
63676,icmp,1
63676,tcp,1
63677,tcp,1
63679,icmp,2
63679,udp,1
6368,udp,1
63680,tcp,1
63683,icmp,2
63683,tcp,1
63685,icmp,1
63685,udp,1
63686,udp,1
63688,udp,2
63689,udp,1
6369,icmp,1
6369,tcp,1
6369,udp,1
63691,icmp,1
63691,tcp,2
63693,icmp,1
63694,icmp,1
63696,udp,1
63697,icmp,1
63697,udp,2
63698,icmp,2
63698,udp,1
63699,icmp,2
63699,tcp,1
63699,udp,1
637,tcp,1
637,udp,1
6370,tcp,1
63701,tcp,2
63701,udp,1
63702,icmp,1
63702,udp,1
63703,icmp,1
63703,tcp,1
63704,tcp,2
63704,udp,1
63706,udp,1
63707,icmp,1
63708,icmp,1
63709,tcp,1
6371,udp,1
63710,udp,1
63711,icmp,3
63711,tcp,2
63711,udp,1
63712,tcp,1
63713,icmp,2
63713,tcp,1
63714,tcp,1
63714,udp,1
63715,icmp,2
63717,tcp,2
63719,tcp,2
6372,icmp,1
6372,tcp,1
6372,udp,1
63720,tcp,1
63720,udp,1
63721,icmp,1
63722,udp,3
63723,icmp,1
63724,icmp,1
63725,tcp,2
63728,udp,2
63729,tcp,1
63

65015,tcp,1
65015,udp,1
65016,udp,2
65017,icmp,1
65017,udp,1
65018,icmp,1
65019,udp,1
6502,udp,2
65020,icmp,1
65020,udp,1
65021,tcp,1
65021,udp,1
65022,tcp,3
65022,udp,2
65024,udp,1
65025,tcp,1
65027,icmp,1
65027,udp,1
65028,icmp,2
65028,udp,1
65029,icmp,1
65029,tcp,2
6503,udp,3
65030,icmp,2
65030,tcp,1
65031,icmp,1
65032,udp,1
65033,tcp,1
65033,udp,3
65035,icmp,1
65036,tcp,1
65037,tcp,1
65038,tcp,1
65038,udp,1
65039,icmp,1
65039,udp,2
6504,icmp,1
65042,icmp,1
65042,udp,1
65045,icmp,2
65045,tcp,1
65045,udp,2
65046,icmp,1
65046,udp,1
65047,udp,1
65049,icmp,1
65050,tcp,2
65053,tcp,1
65053,udp,1
65056,tcp,1
65057,udp,2
65058,tcp,1
65058,udp,1
65059,udp,1
65061,tcp,2
65063,icmp,2
65063,tcp,2
65063,udp,1
65064,udp,3
65065,icmp,1
65067,udp,2
65068,icmp,1
65068,tcp,1
65069,udp,1
65070,icmp,2
65070,udp,1
65071,icmp,1
65072,icmp,1
65072,udp,1
65074,tcp,1
65075,udp,1
65076,tcp,2
65077,icmp,1
65078,tcp,1
65078,udp,1
65079,tcp,1
6508,icmp,1
6508,udp,1
65080,icmp,1
65080,udp,1
65081,tcp,1
65087,udp

7563,udp,1
7564,icmp,1
7564,udp,1
7565,icmp,1
7566,udp,1
7568,icmp,1
7568,tcp,2
7568,udp,1
7570,icmp,1
7570,tcp,1
7570,udp,2
7571,icmp,1
7571,udp,1
7572,udp,1
7573,icmp,1
7573,udp,1
7574,udp,1
7575,udp,1
7576,icmp,1
7576,tcp,1
7576,udp,1
7578,icmp,1
7578,udp,1
7579,udp,1
7580,tcp,1
7581,icmp,2
7581,udp,1
7583,icmp,1
7585,tcp,1
7585,udp,1
7586,tcp,1
7586,udp,1
7587,udp,1
7588,icmp,1
7588,udp,1
7589,icmp,1
759,tcp,1
7590,icmp,1
7591,tcp,2
7592,icmp,1
7593,icmp,1
7595,tcp,1
7595,udp,1
7596,icmp,1
7597,icmp,1
7599,udp,2
76,tcp,1
76,udp,1
760,udp,1
7600,tcp,1
7600,udp,2
7602,tcp,1
7602,udp,1
7603,tcp,1
7604,tcp,1
7605,udp,1
7606,icmp,1
7609,icmp,1
761,tcp,1
7610,icmp,1
7611,tcp,1
7613,udp,1
7614,icmp,1
7615,icmp,1
7615,udp,1
7616,udp,1
7618,icmp,2
7618,udp,2
7620,icmp,1
7621,udp,1
7623,icmp,1
7623,tcp,2
7623,udp,1
7624,icmp,2
7625,tcp,1
7628,icmp,1
7628,tcp,2
7629,icmp,1
7629,tcp,1
7629,udp,2
763,icmp,1
7630,icmp,1
7631,icmp,1
7631,tcp,1
7633,icmp,2
7633,tcp,1
7635,tcp,3
7635,udp,1
7636,icm

8831,udp,1
8834,tcp,1
8836,udp,1
8838,icmp,1
8839,tcp,1
884,udp,1
8840,udp,1
8841,icmp,3
8841,tcp,1
8841,udp,1
8842,udp,2
8844,icmp,1
8847,tcp,1
8848,icmp,1
8848,tcp,1
8849,udp,1
885,icmp,1
885,udp,1
8850,icmp,2
8851,tcp,1
8851,udp,1
8852,icmp,1
8852,udp,1
8853,udp,1
8854,udp,1
8855,icmp,1
8856,udp,2
8858,udp,2
8859,udp,1
886,icmp,1
886,udp,1
8860,icmp,1
8860,tcp,2
8865,udp,1
8867,icmp,1
8868,tcp,2
8868,udp,1
8869,icmp,1
887,udp,1
8870,udp,1
8871,icmp,1
8871,udp,1
8873,udp,2
8874,tcp,1
8876,icmp,1
8877,icmp,1
8878,icmp,1
8879,udp,1
8880,udp,1
8881,icmp,1
8883,tcp,2
8883,udp,1
8884,icmp,2
8884,tcp,2
8884,udp,1
8886,icmp,2
8888,icmp,1
889,icmp,1
889,tcp,1
8891,tcp,2
8892,icmp,2
8893,tcp,1
8894,icmp,1
8897,icmp,3
8898,udp,2
8899,icmp,2
89,icmp,1
89,tcp,1
89,udp,2
890,tcp,1
8900,icmp,1
8900,tcp,2
8901,icmp,1
8901,udp,1
8902,udp,1
8904,icmp,1
8904,udp,1
8905,tcp,2
8907,tcp,1
8908,icmp,1
8908,udp,1
891,icmp,1
8910,icmp,1
8911,icmp,1
8913,tcp,1
8913,udp,1
8914,icmp,1
8914,tcp,3
8915,tcp,1
891